In [73]:
import pandas as pd
import plotly.express as px

In [2]:
org_data = pd.read_parquet("escooter_history_2022.parquet")

### Datencleaning

In [191]:
org_data_clean = org_data.copy()
org_data_clean.holiday = org_data_clean.holiday.astype(bool)
org_data_clean.workingday = org_data_clean.workingday.astype(bool)
#org_data_clean["temp"] = org_data_clean[(org_data_clean.temp == 1394.0)]["temp"].div(100) 

### reg_customer

In [128]:
df_rcus = org_data_clean.copy()
df_rcus = df_rcus.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": "sum"})
df_rcus

,registered_customer
datetime,
2020-01-04,654
2020-01-05,670
2020-01-06,1229
2020-01-07,1454
2020-01-08,1518
...,...
2021-12-30,4024
2021-12-31,7629
2022-01-01,5293


In [126]:
px.line(df_rcus, x=df_rcus.index, y="registered_customer")

### afg 2 und Dateninkonsistenz

In [111]:
df_afg2 = org_data_clean.copy()

In [99]:
df_afg2 = df_afg2[(df_afg2.holiday==1)&(df_afg2.windspeed>35)&(df_afg2.registered_customer==False)]
df_afg2.to_clipboard()

In [107]:
df_afg2_1 = df_afg2.groupby(pd.Grouper(key="datetime",freq="H")).agg({"temp": "mean", "atemp": "mean", "registered_customer": "count"})
df_afg2_2 = df_afg2_1.dropna()
df_afg2_2

,temp,atemp,registered_customer
datetime,,,
2020-09-07 22:00:00,24.60,27.275,18
2020-11-13 10:00:00,13.12,13.635,38
2020-11-13 13:00:00,15.58,19.695,57
2020-12-28 10:00:00,446.90,14.395,51
2020-12-28 11:00:00,13.94,14.395,43


![alt text](s1.png "Dateninkonsistenz")

In [145]:
d_mistake1 = org_data_clean[((org_data_clean.datetime.between("2020-12-27", "2020-12-29"))&(org_data_clean.temp >30))]
d_mistake1.head(5)


,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
1386971,2020-12-28 10:10:06,True,False,"clear, few clouds",1394.0,14.395,42.0,35.0008,False
1386972,2020-12-28 10:10:30,True,False,"clear, few clouds",1394.0,14.395,42.0,35.0008,True
1386973,2020-12-28 10:11:15,True,False,"clear, few clouds",1394.0,14.395,42.0,35.0008,False
1386974,2020-12-28 10:11:40,True,False,"clear, few clouds",1394.0,14.395,42.0,35.0008,True
1386975,2020-12-28 10:11:58,True,False,"clear, few clouds",1394.0,14.395,42.0,35.0008,False


In [157]:
d_mistake2 = org_data_clean[(org_data_clean.temp == 1394.0)]
d_mistake2["temp"] = d_mistake2["temp"] / 100
d_mistake2.head(5)

/var/folders/f6/gs3z1yx9005b5dmpnd_z3t3m0000gn/T/ipykernel_4204/1581765059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_mistake2["temp"] = d_mistake2["temp"] / 100


,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
120382,2020-03-16 10:10:13,False,True,"clear, few clouds",13.94,15.91,42.0,16.9979,False
120383,2020-03-16 10:10:18,False,True,"clear, few clouds",13.94,15.91,42.0,16.9979,True
120384,2020-03-16 10:11:13,False,True,"clear, few clouds",13.94,15.91,42.0,16.9979,True
120385,2020-03-16 10:11:42,False,True,"clear, few clouds",13.94,15.91,42.0,16.9979,False
120386,2020-03-16 10:12:07,False,True,"clear, few clouds",13.94,15.91,42.0,16.9979,False


In [265]:
org_data_clean1 = org_data_clean.copy()

In [288]:
org_data_clean1[(org_data_clean1.temp == 1394.0)].assign(org_data_clean1[(org_data_clean1.temp == 1394.0)].temp.div(100))

TypeError: assign() takes 1 positional argument but 2 were given

In [291]:
org_data_clean1.loc[org_data_clean1.temp == 1394.0, "temp"] = org_data_clean1.temp.div(100)

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer


### afg2

In [293]:
df_afg2_3 = org_data_clean1.copy()

In [294]:
df_afg2_3

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
0,2020-01-04 00:00:09,False,False,"clear, few clouds",9.84,14.395,81.0,0.0000,True
1,2020-01-04 00:00:41,False,False,"clear, few clouds",9.84,14.395,81.0,0.0000,True
2,2020-01-04 00:01:20,False,False,"clear, few clouds",9.84,14.395,81.0,0.0000,True
3,2020-01-04 00:04:12,False,False,"clear, few clouds",9.84,14.395,81.0,0.0000,True
4,2020-01-04 00:15:19,False,False,"clear, few clouds",9.84,14.395,81.0,0.0000,True
...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,False,True,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755411,2022-01-03 23:58:28,False,True,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755412,2022-01-03 23:58:46,False,True,"clear, few clouds",10.66,13.635,65.0,8.9981,True
3755413,2022-01-03 23:59:16,False,True,"clear, few clouds",10.66,13.635,65.0,8.9981,True


In [331]:
df_afg2_4 = df_afg2_3[(df_afg2_3.holiday==1)&(df_afg2_3.windspeed>35)&(df_afg2_3.registered_customer==False)]
df_afg2_5 = df_afg2_4.groupby(pd.Grouper(key="datetime",freq="H")).agg({"temp": "mean", "atemp": "mean", "registered_customer": "count"}).dropna().reset_index()
df_afg2_5


,datetime,temp,atemp,registered_customer
0,2020-09-07 22:00:00,24.60,27.275,18
1,2020-11-13 10:00:00,13.12,13.635,38
2,2020-11-13 13:00:00,15.58,19.695,57
3,2020-12-28 10:00:00,13.94,14.395,51
4,2020-12-28 11:00:00,13.94,14.395,43
